In [4]:
!pip install gdown

  Using cached beautifulsoup4-4.14.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.8.3-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.14.3-py3-none-any.whl (107 kB)
Using cached soupsieve-2.8.3-py3-none-any.whl (37 kB)

   ---------------------------------------- 0/4 [soupsieve]
   ---------- ----------------------------- 1/4 [PySocks]
   -------------------- ------------------- 2/4 [beautifulsoup4]
   -------------------- ------------------- 2/4 [beautifulsoup4]
   -------------------- ------------------- 2/4 [beautifulsoup4]
   ------------------------------ --------- 3/4 [gdown]
   ------------------------------ --------- 3/4 [gdown]
   ------------------------------ --------- 3/4 [gdown]
   ---------------------------------------- 4/4 [gdown]



In [ ]:
import gdown

gdown.download(
    "https://drive.google.com/uc?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu",
    "tennis_court_det_dataset.zip",
    quiet=False
)

In [1]:
!unzip tennis_court_det_dataset.zip

Archive:  tennis_court_det_dataset.zip
   creating: data/
  inflating: data/data_train.json    
  inflating: data/data_val.json      
   creating: data/images/
  inflating: data/images/-0M6ixK7aIU_1050.png  
  inflating: data/images/-0M6ixK7aIU_1100.png  
  inflating: data/images/-0M6ixK7aIU_1150.png  
  inflating: data/images/-0M6ixK7aIU_1200.png  
  inflating: data/images/-0M6ixK7aIU_1250.png  
  inflating: data/images/-0M6ixK7aIU_1300.png  
  inflating: data/images/-0M6ixK7aIU_1350.png  
  inflating: data/images/-0M6ixK7aIU_1500.png  
  inflating: data/images/-0M6ixK7aIU_1550.png  
  inflating: data/images/-0M6ixK7aIU_1600.png  
  inflating: data/images/-0M6ixK7aIU_1650.png  
  inflating: data/images/-0M6ixK7aIU_1700.png  
  inflating: data/images/-0M6ixK7aIU_1900.png  
  inflating: data/images/-0M6ixK7aIU_1950.png  
  inflating: data/images/-0M6ixK7aIU_2000.png  
  inflating: data/images/-0M6ixK7aIU_2050.png  
  inflating: data/images/-0M6ixK7aIU_2100.png  
  inflating: data/images

Start Code

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Create Torch Dataset

In [5]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)

        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [6]:
train_dataset = KeypointsDataset("data/images","data/data_train.json")
val_dataset = KeypointsDataset("data/images","data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

Create Model

In [9]:
model = models.resnet50(pretrained=True)
model.fc =  torch.nn.Linear(model.fc.in_features, 14*2) # Replaces the last layer

In [10]:
model = model.to(device)

Train Model

In [11]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [12]:
epochs=20
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

Epoch 0, iter 0, loss: 14582.025390625
Epoch 0, iter 10, loss: 15909.23046875
Epoch 0, iter 20, loss: 14989.205078125
Epoch 0, iter 30, loss: 13551.8037109375
Epoch 0, iter 40, loss: 13485.26171875
Epoch 0, iter 50, loss: 12763.1728515625
Epoch 0, iter 60, loss: 13269.2861328125
Epoch 0, iter 70, loss: 12461.2509765625
Epoch 0, iter 80, loss: 11950.029296875
Epoch 0, iter 90, loss: 11276.1298828125
Epoch 0, iter 100, loss: 11245.29296875
Epoch 0, iter 110, loss: 11092.01953125
Epoch 0, iter 120, loss: 11351.0712890625
Epoch 0, iter 130, loss: 10352.3125
Epoch 0, iter 140, loss: 9587.0361328125
Epoch 0, iter 150, loss: 9030.0322265625
Epoch 0, iter 160, loss: 9331.7548828125
Epoch 0, iter 170, loss: 8414.478515625
Epoch 0, iter 180, loss: 8223.0966796875
Epoch 0, iter 190, loss: 7940.47802734375
Epoch 0, iter 200, loss: 7535.87158203125
Epoch 0, iter 210, loss: 7686.732421875
Epoch 0, iter 220, loss: 8560.4560546875
Epoch 0, iter 230, loss: 6816.58642578125
Epoch 0, iter 240, loss: 7010

In [14]:
torch.save(model.state_dict(), "keypoints_model.pth")

In [2]:
!nvidia-smi

Fri Jan 30 22:51:27 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 581.80                 Driver Version: 581.80         CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   53C    P0             14W /   38W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----